# Import

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/[2023-2024] AN2DL/Homework2

In [ ]:
import numpy as np
import logging
import random

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import mixed_precision

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Conv1D, Cropping1D, Dropout, Dense, GlobalAveragePooling1D, Reshape, Concatenate, BatchNormalization, Activation, Add, MaxPooling1D
from tensorflow.keras.models import Model


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

Fix randomness and hide warnings


In [ ]:
seed = np.random.randint(0, 1000)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)
print("Random Seed:",seed);

In [ ]:
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Data processing

In [ ]:
trainingSetLoad = "Datasets/training_data.npy"
periodsLoad = "Datasets/valid_periods.npy"
labelsLoad = "Datasets/categories.npy"
unzip = False

if unzip:
    !unzip training_dataset.zip

trainingSet = np.load(trainingSetLoad, allow_pickle=True)
periods = np.load(periodsLoad, allow_pickle=True)
categories = np.load(labelsLoad, allow_pickle=True)

Create an array containing the time series without the padding. The len of such array in fact is equal to the number of time series provided

In [ ]:
noPadding_data = []
dataShape = trainingSet.shape
for i in range(dataShape[0]):
  noPadding_data.append(trainingSet[i,periods[i,0]:periods[i,1]])

new_data = np.array(noPadding_data)
print(new_data.shape)

## Build windows
Here you can find the function that creates the windows given the training set, the window size, the stride and the telescope (how many steps to predict). Such function also returns an array *sources* that contains the category of each window. \\
In the innermost *for* loop you can also find an *if* condition, which specifies the least amount of samples required for a window to have in order to be used

In [ ]:
def build_sequences(df, window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    source = []
    for i in range(len(df)):
      assert window % stride == 0

      temp_df = df[i].copy()
      temp_label = df[i].copy()
      padding_check = len(df[i]) % window

      if(padding_check != 0):
          # Compute padding length
          padding_len = window - len(df[i]) % window
          padding = np.zeros((padding_len), dtype='float32')
          temp_df = np.concatenate((padding,temp_df))
          padding = np.zeros((padding_len), dtype='float32')
          temp_label = np.concatenate((padding,temp_label))
          assert len(temp_df) % window == 0

      for idx in np.arange(0,len(temp_df)-window-telescope,stride):
          # if(not np.all(temp_df[idx:idx+window] == 0)):
          if np.count_nonzero(temp_df[idx:idx+window]) >= (window / 2):
            dataset.append(temp_df[idx:idx+window])
            labels.append(temp_label[idx+window:idx+window+telescope])
            source.append(categories[i])

    dataset = np.array(dataset)
    labels = np.array(labels)
    source = np.array(source)
    return dataset, labels, source

Build the set of windows, the related predictions, and the array of categories for each window

In [ ]:
window = 200
stride = 10
telescope = 18

X, Y, sources = build_sequences(new_data, window, stride, telescope)

X.shape, Y.shape, sources.shape


In [ ]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, Y, test_size=0.1, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, shuffle=True)

X_train.shape, X_val.shape, y_val.shape, y_train.shape

Inspect some of the windows created

In [ ]:
n_images = 6
figs, axs = plt.subplots(n_images, 1, sharex=True, figsize=(14,14))
for i in range(n_images):
  idx = np.random.randint(len(X_train))
  axs[i].plot(X_train[idx])
plt.show()

# Robust Scalers

Apply Robust Scalers to the data (has only been used for the *LSTMs concatenated* model)




In [ ]:
from sklearn.preprocessing import RobustScaler

# Apply robust scaling (fit only to training data to avoid bias)
rscaler_X = RobustScaler().fit(X_train)
rscaler_y = RobustScaler().fit(y_train)
X_train = rscaler_X.transform(X_train)
X_val = rscaler_X.transform(X_val)
scalerX = rscaler_X.get_params()
scalerY = rscaler_y.get_params()
print("Scaler X",scalerX)
print("Scaler Y",scalerY)
y_train = rscaler_y.transform(y_train)
y_val = rscaler_y.transform(y_val)

# Models

# Stacked LSTMs model

In [ ]:
dropout_rate = 0.2
lstm_units = 128

def build_stacked_lstm_model(input_shape, lstm_units=128, dropout_rate=0.2, forecast_length=18):
    input_layer = tfkl.Input(shape=input_shape)
    # first block
    x = tfkl.Bidirectional(
        tfkl.LSTM(units=lstm_units, return_sequences=True, kernel_regularizer=tfk.regularizers.l2(0.001)))(input_layer)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    # second block
    x = tfkl.LSTM(units=lstm_units // 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    x = tfkl.Dense(units=forecast_length)(x)

    model = tfk.Model(inputs=input_layer, outputs=x)

    model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=tf.keras.optimizers.Adam(), metrics=["mse"])


    return model

## Convolutional LSTM

In [ ]:
def CONV_LSTM(input_shape, output_shape):

    # Define the input layer with the specified shape
    input_layer = tfkl.Input(shape=input_shape)

    x = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True))(input_layer)
    conv = tfkl.Conv1D(128,20, padding="same", activation="relu")(x)
    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(conv)
    conv = tfkl.Conv1D(64,10, padding="same", activation="relu")(x)
    x = tfkl.Bidirectional(tfkl.LSTM(32, return_sequences=True))(conv)

    # Add a final Convolution layer to match the desired output shape
    output_layer = tfkl.Conv1D(output_shape[1], 3, padding='same')(x)

    # Calculate the size to crop from the output to match the output shape
    crop_size = output_layer.shape[1] - output_shape[0]

    # Crop the output to the desired length
    output_layer = tfkl.Cropping1D((0, crop_size), name='cropping')(output_layer)

    # Construct the model by connecting input and output layers
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM')

    model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=tf.keras.optimizers.Adam(), metrics=["mse"])

    return model

## Autoencoder with LSTM
One note to add on this model is that we trained it with a *weight_decay* equal to the default learning rate *(0.001)*

In [ ]:
def decoder_block(prev, filters, kernel_size):

  x = tfkl.UpSampling1D()(prev)
  x = tfkl.Conv1D(filters, kernel_size, padding="same")(x)
  x = tfkl.BatchNormalization()(x)
  x = tfkl.LeakyReLU()(x)

  return x

def encoder_block(prev, filters, kernel_size):

  x = tfkl.Conv1D(filters, kernel_size, padding="same")(prev)
  x = tfkl.BatchNormalization()(x)
  x = tfkl.LeakyReLU()(x)
  x = tfkl.MaxPooling1D()(x)

  return x

def AUTOENCODER_LSTM(input_shape ,output_shape):

    input_layer = tfkl.Input(shape=input_shape)

    ## ENCODER
    x = encoder_block(input_layer, 32, 13)
    x = tfkl.Bidirectional(tfkl.LSTM(16, return_sequences=True, dropout=0.2))(x)
    x = encoder_block(x, 64, 8)
    x = tfkl.Bidirectional(tfkl.LSTM(32, return_sequences=True, dropout=0.2))(x)
    x = encoder_block(x, 128, 3)
    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, dropout=0.2))(x)
    # x.output_shape = (None,25,128)
    x = tfkl.Flatten()(x)
    x = tfkl.Dense(9)(x)

    ## DECODER
    # The shape (25,128) is the output shape of the last Bidirectional LSTM of the encoder
    x = tfkl.Dense(25*128)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.LeakyReLU()(x)
    # Rebuild the shape produced by the encoder
    x = tfkl.Reshape((25,128))(x)

    x = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True, dropout=0.2))(x)
    x = decoder_block(x, 64, 13)
    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, dropout=0.2))(x)
    x = decoder_block(x, 32, 8)
    x = tfkl.Bidirectional(tfkl.LSTM(32, return_sequences=True, dropout=0.2))(x)
    x = decoder_block(x, 16, 3)

    x = tfkl.GlobalAveragePooling1D()(x)
    output_layer = tfkl.Dense(18)(x)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='AUTOENCODER_LSTM')

    model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=tf.keras.optimizers.Adam(weight_decay=0.001), metrics=["mse"])

    return model

## Sequence To Sequence with Luong Attention

In [ ]:
def build_lstm_seq2seq_attention(input_shape,n_units=128):


    input_layer = tfkl.Input(shape=input_shape, name='Input')

    encoder_x,encoder_h,encoder_c = tfkl.LSTM(units=n_units, return_sequences=True,return_state=True)(input_layer)

    decoder_in = tfkl.RepeatVector(1)(encoder_h)

    x = tfkl.LSTM(units=n_units, return_sequences=True,return_state=False)(decoder_in,initial_state=[encoder_h,encoder_c])
    decoder_x = tfkl.Bidirectional(tfkl.LSTM(units=int(n_units/2), return_sequences=True,return_state=False))(x)

    attention = tfkl.Dot(axes=[2,2])([decoder_x, encoder_x])
    attention = tfkl.Activation('softmax')(attention)
    context = tfkl.Dot(axes=[2,1])([attention,encoder_x])

    concatenated_c = tfkl.Concatenate()([context,decoder_x])
    concatenated_c = tfkl.Flatten()(concatenated_c)
    output_layer = tfkl.Dense(18)(concatenated_c)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='s2s_Attention')

    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])

    return model

## WaveNet

In [ ]:
def build_wavenet(input_shape):

    input_layer = tfkl.Input(shape=input_shape, name='Input')

    x = tfkl.Conv1D(filters=16, kernel_size=3, dilation_rate=1, padding="causal", activation='relu')(input_layer)
    x = tfkl.Conv1D(filters=32, kernel_size=3, dilation_rate=2, padding="causal", activation='relu')(x)
    x = tfkl.Conv1D(filters=64, kernel_size=3, dilation_rate=4, padding="causal", activation='relu')(x)
    x = tfkl.Conv1D(filters=128, kernel_size=3, dilation_rate=8, padding="causal", activation='relu')(x)
    x = tfkl.Conv1D(filters=256, kernel_size=3, dilation_rate=16, padding="causal", activation='relu')(x)
    x = tfkl.BatchNormalization()(x)
    x1 = tfkl.MaxPool1D()(x)

    x1,h1,c1 = tfkl.LSTM(units=256,return_sequences=True, return_state=True)(x1)

    x = tfkl.Conv1D(filters=16, kernel_size=3, dilation_rate=1, padding="causal", activation='relu')(x1)
    x = tfkl.Conv1D(filters=32, kernel_size=3, dilation_rate=2, padding="causal", activation='relu')(x)
    x = tfkl.Conv1D(filters=64, kernel_size=3, dilation_rate=4, padding="causal", activation='relu')(x)
    x = tfkl.Conv1D(filters=128, kernel_size=3, dilation_rate=8, padding="causal", activation='relu')(x)
    x = tfkl.Conv1D(filters=256, kernel_size=3, dilation_rate=16, padding="causal", activation='relu')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Add()([x1, x])
    x1 = tfkl.MaxPool1D()(x)

    x1,h1,c1 = tfkl.LSTM(units=256, return_sequences=True, return_state=True)(x1, initial_state=[h1, c1])

    x = tfkl.Conv1D(filters=16, kernel_size=3, dilation_rate=1, padding="causal", activation='relu')(x1)
    x = tfkl.Conv1D(filters=32, kernel_size=3, dilation_rate=2, padding="causal", activation='relu')(x)
    x = tfkl.Conv1D(filters=64, kernel_size=3, dilation_rate=4, padding="causal", activation='relu')(x)
    x = tfkl.Conv1D(filters=128, kernel_size=3, dilation_rate=8, padding="causal", activation='relu')(x)
    x = tfkl.Conv1D(filters=256, kernel_size=3, dilation_rate=16, padding="causal", activation='relu')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Add()([x1, x])
    x1 = tfkl.MaxPool1D()(x)

    x1,h1,c1 = tfkl.LSTM(units=256, return_sequences=False, return_state=True)(x1, initial_state=[h1, c1])

    x = tfkl.Flatten()(x1)
    x = tfkl.Dense(256, activation='relu')(x)
    x = tfkl.Dropout(.2)(x)
    output_layer = tfkl.Dense(18)(x)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(), metrics=['mae'])

    return model

## ResNet with LSTM

In [ ]:
def residual_block(x, filters=64, kernel_size=3, stride=1):
      x = tfkl.Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
      shortcut = x
      x = tfkl.BatchNormalization()(x)
      x = tfkl.Activation('relu')(x)
      x = tfkl.Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
      x = tfkl.BatchNormalization()(x)
      x = tfkl.Activation('relu')(x)
      x = tfkl.Add()([x, shortcut])
      return x

def build_resNet_lstm_model(input_shape, output_shape):

    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    x = tfkl.Bidirectional(LSTM(100, return_sequences=True, name='lstm'), name='bidirectional_lstm')(input_layer)
    x = tfkl.Conv1D(128, 3, strides=2, padding='same')(x)
    x = tfkl.MaxPooling1D(pool_size=2, padding="valid")(x)

    #ResNet Block
    x = residual_block(x,filters=64)
    x = residual_block(x,filters=64)
    x = residual_block(x,filters=64)
    x = tfkl.GlobalAveragePooling1D()(x)
    x = tfkl.Dense(18)(x)
    x = tfkl.Reshape((18,1))(x)
    output_layer = tfkl.Bidirectional(tfkl.LSTM(9, return_sequences=True, name='lstm'), name='bidirectional_lstm_2')(x)
    output_layer = tfkl.Conv1D(1, 3, padding='same', name='output_layer')(output_layer)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(), metrics=['mae'])

    return model

# Training

Build the model, print the summary and plot the model


In [ ]:
x = np.expand_dims(X_train, axis=2)
y = np.expand_dims(y_train, axis=2)

input_shape = x.shape[1:]
output_shape = y.shape[1:]

print(input_shape, output_shape)
model = build_stacked_lstm_model(input_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

Start the training

In [ ]:
batch_size = 256
epochs = 150
history = model.fit(
    x = x,
    y = y,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val, y_val),
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=8, factor=0.2, min_lr=1e-5),
        tfk.callbacks.ModelCheckpoint(
            filepath='weights/model-name.{epoch:02d}-{val_loss:.4f}.h5',
            save_freq='epoch', verbose=1, monitor='val_loss',
            save_weights_only=True,
        )
    ]
).history

# Predict

In [ ]:
predictions = model.predict(X_test, verbose=0)

# In some models the output shape is the following (BS, 9 or 18, 1) because of the use of a Conv1D as output layer
# the following line of code is used to remove that last dimension and to translate the prediction shape to (BS, 9 or 18)
# predictions = predictions.reshape((predictions.shape[0], -1))[:,:9]

In the following cell there is the code to implement the prediction with an *autoregressive* approach as explained in the report: \\
predict 18 steps in the future in 2 rows, 9 samples then another 9 samples (we are assuming the model used to predict has an output layer length of 9). \\

Note that this is an example: when uploading a submission, the *model.py* contains this code below but slightly adapted to work with the *model* class (*model.predict* -> *self.model.predict* and so on)

In [ ]:
1st_predictions = model.predict(X_test, verbose=0)
# 1st_predictions = 1st_predictions.reshape((1st_predictions.shape[0], -1))[:,:9]

# Shift the input windows of 9 time steps forward and include the 9 samples we just predicted
nextWindow = np.concatenate((X_test[:,9:], 1st_predictions), axis=1)

2nd_predictions = model.predict(nextWindow, verbose=0)
# 2st_predictions = 2st_predictions.reshape((2st_predictions.shape[0], -1))[:,:9]

predictions = np.concatenate((1st_predictions, 2st_predictions), axis=1)

## View results

Compute the MSE and MAE metrics

In [ ]:
# Calculate and print Mean Squared Error (MSE)
mean_squared_error = tfk.metrics.mean_squared_error(y_test.flatten(), predictions.flatten()).numpy()
print(f"Mean Squared Error: {mean_squared_error}")

# Calculate and print Mean Absolute Error (MAE)
mean_absolute_error = tfk.metrics.mean_absolute_error(y_test.flatten(), predictions.flatten()).numpy()
print(f"Mean Absolute Error: {mean_absolute_error}")